# Working with NWB in Python

**Important**: This notebook will only work with the 2.10.0 version of the h5py package. The cell below will ensure that you have this version installed. If not, you should run the `!pip install` line.

In [1]:
# This will ensure that the correct version of the h5py package is installed
try:
    import h5py
    if h5py.__version__ == '2.10.0':
         print('h5py version ' + h5py.__version__ + ' already installed')
    else:
        print('h5py installed with an older version. some features may not work.')
except ImportError as e:
    !pip install h5py == '2.10.0'

h5py version 2.10.0 already installed


In [2]:
# Import any necessary packages
from pynwb import NWBHDF5IO

# read the NWB file
io = NWBHDF5IO('000006/sub-anm369962/sub-anm369962_ses-20170310.nwb', 'r')
nwb_file = io.read()
print('File downloaded.')
print(type(nwb_file))

File downloaded.
<class 'pynwb.file.NWBFile'>


## File Hierarchy: Groups, Datasets, and Attributes¶

The NWB file is composed of various Groups, Datasets, and Attributes. The data/datasets and cooresponding meta-data are encapsulated within these Groups. The `fields` attribute returns a dictionary contiaining the the Groups of our nwb file. The dictionary keys are the various Groups within the file which we will use to access our datasets.

In [3]:
# Get the Groups for the nwb file 
nwb_fields = nwb_file.fields
print(nwb_fields.keys())

dict_keys(['acquisition', 'analysis', 'scratch', 'stimulus', 'stimulus_template', 'processing', 'devices', 'electrode_groups', 'imaging_planes', 'icephys_electrodes', 'ogen_sites', 'intervals', 'lab_meta_data', 'session_description', 'identifier', 'session_start_time', 'timestamps_reference_time', 'file_create_date', 'keywords', 'epoch_tags', 'electrodes', 'subject', 'trials', 'units', 'experiment_description', 'institution', 'experimenter', 'related_publications'])


Each NWB file will have information on where the experiment was conducted, what lab conducted the experiment, as well as a description of the experiment. These Groups can be accessed using `institution`, `lab`, and `experiment_description`, attributes on our nwb_file, respectively.

In [4]:
# Get Meta-Data from NWB file 
print('The experiment within this NWB file was conducted at {} in the lab of {}. The experiment is detailed as follows: {}'.format(nwb_file.institution, nwb_file.lab, nwb_file.experiment_description))

The experiment within this NWB file was conducted at Janelia Research Campus in the lab of None. The experiment is detailed as follows: Extracellular electrophysiology recordings performed on mouse anterior lateral motor cortex (ALM) in delay response task. Neural activity from two neuron populations, pyramidal track upper and lower, were characterized, in relation to movement execution.


We can access datasets from each group in our nwb_file with the following syntax: `nwb_file.group`. This is no different than executing a method and/or attribute. Below we will demonstrate some of the useful groups within an `NWBFile` object. 

The `acquisition` group contains datasets of acquisition data, mainly `TimeSeries` objects belonging to this NWBFile. 

In [5]:
nwb_file.acquisition

{'lick_times': lick_times pynwb.behavior.BehavioralEvents at 0x140492891752272
 Fields:
   time_series: {
     lick_left_times <class 'pynwb.base.TimeSeries'>,
     lick_right_times <class 'pynwb.base.TimeSeries'>
   }}

In this file, the acquisition group contains two different `Fields`, `lick_left_times` and `lick_right_times` within the `lick_times` dataset. To access the actual data arrays of these fields we must first subset our dataset of interest from the group. We can then use `data[:]` to return our actual data array.

In [6]:
# select our dataset of interest 
dataset = 'lick_times'
field = 'lick_right_times'
lick_r_dataset = nwb_file.acquisition[dataset][field]

# return first 10 values in data array 
lick_r_data_array = lick_r_dataset.data[:10]

print(lick_r_data_array)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


The `intervals` group contains all time interval tables from the experiemnt. We can look at the `description` field to understand what each dataset in contains.

In [7]:
# example showing how to return meta data from groups in nwb file 
nwb_file.intervals

{'trials': trials pynwb.epoch.TimeIntervals at 0x140492892058320
 Fields:
   colnames: ['start_time' 'stop_time' 'type' 'response' 'stim_present' 'is_good'
  'cue_start_time' 'pole_in_time' 'pole_out_time']
   columns: (
     start_time <class 'hdmf.common.table.VectorData'>,
     stop_time <class 'hdmf.common.table.VectorData'>,
     type <class 'hdmf.common.table.VectorData'>,
     response <class 'hdmf.common.table.VectorData'>,
     stim_present <class 'hdmf.common.table.VectorData'>,
     is_good <class 'hdmf.common.table.VectorData'>,
     cue_start_time <class 'hdmf.common.table.VectorData'>,
     pole_in_time <class 'hdmf.common.table.VectorData'>,
     pole_out_time <class 'hdmf.common.table.VectorData'>
   )
   description: experimental trials
   id: id <class 'hdmf.common.table.ElementIdentifiers'>}

Within the intervals group is the `trials` dataset which is a `DynamicTable` contianing intervals from our experimental trials. Each column in `trials` is a `VectorData` object which can all be assigned to a dataframe using `to_dataframe()`.

In [8]:
# Select the group of interest from the nwb file 
intervals = nwb_file.intervals

# Subset the dataset from the group and assign it as a dataframe
interval_trials_df = intervals['trials'].to_dataframe()
interval_trials_df.head()

start_time  stop_time        type    response  stim_present  is_good  \
id                                                                         
1      323.171        NaN  lick right  early lick             0        1   
2      329.570        NaN  lick right  early lick             0        1   
3      335.971        NaN  lick right   incorrect             0        1   
4      342.371        NaN  lick right   incorrect             0        1   
5      348.289        NaN  lick right   incorrect             0        1   

    cue_start_time  pole_in_time  pole_out_time  
id                                               
1          7.49698       3.93949        5.12748  
2         11.81280       4.94181        5.94198  
3          6.62964       4.12915        5.12931  
4          6.41964       3.91915        4.91931  
5         12.66990      10.16930       11.16950

The `description` attribute provides a short description on each column of the dataframe.

In [9]:
# return the description of each col in our dataframe
for col in intervals['trials'].to_dataframe():
    print(col +':')
    print(intervals['trials'][col].description)
    print('\n')

start_time:
Start time of epoch, in seconds


stop_time:
Stop time of epoch, in seconds


type:



response:



stim_present:
is this a stim or no-stim trial


is_good:
good/bad status of trial (bad trials are not analyzed)


cue_start_time:
onset of response period


pole_in_time:
onset of sample period


pole_out_time:
onset of the delay period




The `units` group in our nwb_file contains the neural activity of our units, including spike data for scientific analysis. Much like the `intervals` group, `units` is also a `DynamicTable` that can be assigned to a dataframe.

In [10]:
units = nwb_file.units
units_df = units.to_dataframe()
units_df.head()

depth quality     cell_type  \
id                                
1   665.0    Poor  unidentified   
2   665.0    Fair  unidentified   
3   715.0          unidentified   
4   715.0      []  unidentified   
5   715.0    Fair  unidentified   

                                          spike_times  \
id                                                      
1   [933.873288, 948.2774710000008, 950.5357110000...   
2   [329.95417899999956, 330.01945899999953, 330.0...   
3   [329.94165899999956, 329.9998989999996, 330.00...   
4   [329.93145899999956, 330.7492189999995, 330.77...   
5   [331.09961899999956, 332.14505899999955, 333.3...   

                                           electrodes  
id                                                     
1           x      y      z     imp  \
id         ...  
2           x      y      z     imp  \
id         ...  
3           x      y      z     imp  \
id         ...  
4           x      y      z     imp  \
id         ...  
5           x      y      z     imp  \
id         ...

The `electrodes` group contians metadata from the elctrodes used in the experimental trials. Also a `DynamicTable`, the data includes location of the electrodes, type of filtering, and the whats electrode group the electrode belongs to. 

In [11]:
# electrode positions 
electrodes = nwb_file.electrodes
electrodes_df = electrodes.to_dataframe()
electrodes_df.head()

x    y    z  imp                                           location  \
id                                                                          
1   0.0  0.0  0.0 -1.0  brain_region: ALM; brain_subregion: N/A; corti...   
2   0.0  0.0  0.0 -1.0  brain_region: ALM; brain_subregion: N/A; corti...   
3   0.0  0.0  0.0 -1.0  brain_region: ALM; brain_subregion: N/A; corti...   
4   0.0  0.0  0.0 -1.0  brain_region: ALM; brain_subregion: N/A; corti...   
5   0.0  0.0  0.0 -1.0  brain_region: ALM; brain_subregion: N/A; corti...   

                      filtering  \
id                                
1   Bandpass filtered 300-6K Hz   
2   Bandpass filtered 300-6K Hz   
3   Bandpass filtered 300-6K Hz   
4   Bandpass filtered 300-6K Hz   
5   Bandpass filtered 300-6K Hz   

                                                group group_name  
id                                                                
1   H-129: 64 pynwb.ecephys.ElectrodeGroup at 0x14...  H-129: 64  
2   H-129: 64 pynwb.ecephys.ElectrodeGroup at 0x14...  H-129: 64  
3   H-129: 64 pynwb.ecephys.ElectrodeGroup at 0x14...  H-129: 64  
4   H-129: 64 pynwb.ecephys.ElectrodeGroup at 0x14...  H-129: 64  
5   H-129: 64 pynwb.ecephys.ElectrodeGroup at 0x14...  H-129: 64

## Additional Resources 

For an in depth explanation of all groups contained within an `NWBFile` object please visit the <a href = 'https://pynwb.readthedocs.io/en/stable/pynwb.file.html'> pynwb.file.NWBFile </a> section of the PyNWB documentation. 